In [8]:
import ipyvolume as ipv
import numpy as np

In [15]:
from typing import Iterable, List
from collections import namedtuple
import csv
import math
from io import TextIOWrapper
from zipfile import ZipFile


PC_TO_LIGHT_YEAR = 0.3066014

System = namedtuple('System', 
    ['hab_hyg_idx', 'hip_idx', 'is_hab', 'display_name', 'Hyg_idx', 
     'bf_name', 'gliese_idx', 'bd_idx', 'hd_idx', 'hr_idx', 'proper_name', 
     'spectral_class', 'distance', 'x', 'y', 'z', 'abs_mag'])


def load_starmap():
    systems = {}
    with ZipFile('HabHYG.zip') as zf:
        with zf.open('HabHYG.csv', 'r') as file:
            reader = csv.reader(TextIOWrapper(file, 'iso-8859-1'))
            for i, row in enumerate(reader):
                if i != 0:
                    assert row[0] not in systems.keys()
                    systems[row[0]] = System(*row)
    return systems


def dist_sqr(pos1: Iterable[float], pos2: Iterable[float]) -> float:
    return sum(math.pow(p2 - p1, 2.0) for p1, p2 in zip(pos1, pos2))


def dist(pos1: Iterable[float], pos2: Iterable[float]) -> float:
    return math.sqrt(dist_sqr(pos1, pos2))


def make_3d_point(sys: System):
    return (float(sys.x), float(sys.y), float(sys.z))


def dist_sqr_between_systems(sys1: System, sys2: System):
    return dist_sqr(make_3d_point(sys1), make_3d_point(sys2))


def get_systems_within(
        current_system_idx: str, systems: List[System], max_distance: float)\
        -> List:
    current_system = systems[current_system_idx]
    threshold = math.pow(max_distance, 2)
    filter_list = []
    for _, system in systems.items():
        if dist_sqr_between_systems(current_system, system) <= threshold:
            #print(f"{system.display_name}:\t{system.spectral_class}\t{round(float(system.abs_mag), 2)}")
            filter_list.append((float(system.x), float(system.y), float(system.z)))
    return filter_list

In [24]:
systems = load_starmap()
coords = get_systems_within('0', systems, 20)
x, y, z = zip(*coords)

In [26]:
ipv.quickscatter(np.asarray(x), np.asarray(y), np.asarray(z), size=0.1, marker="sphere")